In [1]:
import fiona
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

import geopandas as gpd
from shapely.geometry import Point, Polygon

import zipfile
import requests
import os
import shutil

from sklearn.preprocessing import *

from lnks import scl_cols

#import tensorflow as tf

%matplotlib inline

import warnings #DANGER: I triggered a ton of warnings.
warnings.filterwarnings('ignore')

np.random.seed()

from statistics import mean, median

# To start...

We import the data output of our data pipeline. We reset the index, drop index columns, and lag the data.

We explicitly print shape several times, making sure that we capture the magnitude of data lost from dropping NA values.

In [4]:
#Shift and shape vars
shiftmonths = 6
shapef = 'ward_anc'
#Assign the split for holdout data.
holdout_date = 2015.5
#Get data
filestring = './data/'+shapef+'_out.csv'
df = pd.read_csv(filestring)
df = df.sort_values(['month', 'NAME'])# , 'ANC'])
df = df.reset_index(drop=True)
len(df.NAME.unique())

48

Now we examine the columns and lag the data.

In [5]:
df.columns

Index(['Unnamed: 0', 'NAME', 'Util_Indx_BBL', 'countBBL', 'countIssued',
       'month', 'SALEPRICE', 'Q_GDP', 'BIZ_Dist_Concentr',
       'GS_GRANTS_Concentr', 'LIQUOR_Concentr', 'PHARM_Concentr',
       'GROC_Concentr', 'BANKS_Concentr', 'CLUBS_Concentr', 'HOTELS_Concentr',
       'METRO_Concentr', 'pct_metro_coverage'],
      dtype='object')

In [6]:
print(df.shape)
shiftnum= (((len(df.NAME.unique()))*(shiftmonths)))

#Also generate some lagged y data in the opposite direction.
df['y']= df['countBBL'].shift(-shiftnum)
df['countBBL_prev_month'] = df['countBBL'].shift((len(df.NAME.unique())))
df['countBBL_prev_cycle'] = df['countBBL'].shift((shiftnum))
df = df[shiftnum:-(shiftnum+(len(df.NAME.unique())))]
df = df.dropna()
df.shape

(2880, 18)


(2182, 21)

The next cell cleans out vestigial columns and drops/fills/expands to dummies for our NA and categorical values.

In [7]:
df = pd.get_dummies(df, columns=['NAME'])
df = df.drop(['Unnamed: 0'], axis= 1)
print(df.shape)
df = df.astype('float')

df = df.dropna()
print(df.shape)

(2182, 67)
(2182, 67)


Here we start building our grid search inputs, beginning with the splits.

In [8]:
#Flexible adaptation of Dr. Braman's interactive gridsearch script
#implementation. 
#TODO Clean up and streamline
import sklearn
from sklearn.neural_network import *
from sklearn.neighbors import *
from sklearn.svm import *
from sklearn.gaussian_process import *
from sklearn.gaussian_process.kernels import *
from sklearn.tree import *
from sklearn.ensemble import *
from sklearn.naive_bayes import *
from sklearn.discriminant_analysis import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
import random

#Frame up some separate DataFrames for scalar and stuff
scl_data = data = df



data     = data.reset_index(drop=True)
X = data.drop(['y'], axis=1)
y = data['y']

XH_train = data[data['month'] <= holdout_date-1]
yH_train = XH_train['y']
XH_train = XH_train.drop(['y'], axis=1)

XH_val = scl_data[scl_data['month'] >= holdout_date-1]
XH_val = XH_val[XH_val['month'] <= holdout_date]

yH_val = XH_val['y']
XH_val = XH_val.drop(['y'], axis=1)

XH_test  = data[data['month'] >= holdout_date]
yH_test  = XH_test['y']
XH_test  = XH_test.drop(['y'], axis=1)

ytr = sklearn.preprocessing.MinMaxScaler([0, 1]
            ).fit(y)
y = ytr.fit_transform(y)
y = pd.DataFrame(y, columns=['y'])

scl_data = scl_data.reset_index(drop=True)

In [9]:
y.y

0       0.030214
1       0.041713
2       0.035841
3       0.008930
4       0.108869
5       0.055046
6       0.001346
7       0.039021
8       0.039388
9       0.022141
10      0.017615
11      0.014190
12      0.010275
13      0.006606
14      0.011743
15      0.026667
16      0.025688
17      0.014679
18      0.008073
19      0.013700
20      0.034251
21      0.028991
22      0.035352
23      0.031927
24      0.042202
25      0.037309
26      0.028379
27      0.019205
28      0.020795
29      0.010398
          ...   
2152    0.087584
2153    0.059205
2154    0.050642
2155    0.061774
2156    0.127217
2157    0.108746
2158    0.145810
2159    0.125994
2160    0.171498
2161    0.145443
2162    0.125505
2163    0.115229
2164    0.068746
2165    0.052477
2166    0.059694
2167    0.057615
2168    0.042569
2169    0.075963
2170    0.034985
2171    0.062263
2172    0.040734
2173    0.055780
2174    0.473884
2175    1.000000
2176    0.380428
2177    0.320000
2178    0.508379
2179    0.6957

In [10]:
print(scl_data.month.max())
print(scl_data.shape)
scl_data = scl_data.dropna()
print(scl_data.shape)
sXH_train = scl_data[scl_data['month'] <= holdout_date-1]
syH_train = sXH_train['y']
sXH_train = sXH_train.drop(['y'], axis=1)

sXH_val = scl_data[scl_data['month'] >= holdout_date-1]
sXH_val = sXH_val[sXH_val['month'] <= holdout_date]

syH_val = sXH_val['y']
sXH_val = sXH_val.drop(['y'], axis=1)


sXH_test  = scl_data[scl_data['month'] >= holdout_date]
syH_test  = sXH_test['y']
sXH_test  = sXH_test.drop(['y'], axis=1)

2016.05
(2182, 67)
(2182, 67)


In [11]:
#Build scalers for the scl_data, other --------------------
scale_data_splits = [scl_data, sXH_train,sXH_test, syH_train, syH_test]
for scl_data in scale_data_splits:
    scaler = sklearn.preprocessing.StandardScaler(
                ).fit(scl_data)
    minmaxer = sklearn.preprocessing.MinMaxScaler([0, 1]
                ).fit(scl_data)

    scl = scaler.transform(scl_data)
    scl = minmaxer.transform(scl_data)
    try:
        scl_data = pd.DataFrame(scl, columns=scl_data.columns)
    except AttributeError as e:
        print(e)
        scl_data = pd.DataFrame(scl, columns=['y'])
    print(scl_data.shape)
    scl_data = scl_data.dropna()
    print(scl_data.shape)
    assert np.all(np.isfinite(scl_data))
    assert not np.any(np.isnan(scl_data))
    
    
#scl_data[scl_data.columns
#   ] = scaler.fit_transform(scl_data[scl_data.columns])

#----------------------------------------------------------


(2182, 67)
(2182, 67)
(1393, 66)
(1393, 66)
(232, 66)
(232, 66)
'Series' object has no attribute 'columns'
(1393, 1)
(1393, 1)
'Series' object has no attribute 'columns'
(232, 1)
(232, 1)


Let's make sure our data came out of the scalers intact:

In [12]:
y;

In [13]:
print(sXH_train.shape)
print(syH_train.shape)
print(sXH_test.shape)
print(syH_test.shape)


(1393, 66)
(1393,)
(232, 66)
(232,)


In [14]:
scl_data.columns

Index(['y'], dtype='object')

In [15]:
sX = scl_data.drop(['y'], axis=1)
sy = scl_data['y']



assert np.all(np.isfinite(X))
assert np.all(np.isfinite(y))
assert not np.any(np.isnan(X))
assert not np.any(np.isnan(y))

assert np.all(np.isfinite(sX))
assert np.all(np.isfinite(sy))
assert not np.any(np.isnan(sX))
assert not np.any(np.isnan(sy))

In [16]:
scl_data.columns

Index(['y'], dtype='object')

In [17]:
scl_data.describe()

,y
count,232.000000
mean,0.144699
std,0.176489
min,0.000000
25%,0.042825
50%,0.080460
75%,0.138425
max,1.000000


This cell contains our a crude RNG, a list of regressors which benefit from scaled data, and hardcoded data used to generate our param_grid, et cetera.

In [18]:
#Make a short list of random states to insert into randomstate params.
scrambler = []
for scram in range(0, 10):
    scrambler.append(random.randint(0, 10000))   
print(scrambler)

to_scale = ['SVR']

names       = ['AdaBoostRegressor',
             'RandomForestRegressor',
             'SVR',
             #'KNeighborsRegressor',
             #'BaggingRegressor',
             'GradientBoostingRegressor',
             #'LinearRegression',
             #'MLPRegressor',
             #'SGDRegressor',
             'LassoLars'         
    
]

regressors = [AdaBoostRegressor(),
              RandomForestRegressor(),
              SVR(),
              #KNeighborsRegressor(),
              #BaggingRegressor(),
              GradientBoostingRegressor(),
              #LinearRegression(),
              #MLPRegressor(),
              #SGDRegressor(),
              LassoLars()
    
]

param_grids =[ 
    ['AdaBoostRegressor', dict(
        n_estimators=[80, 60, 30],
        learning_rate=[1, .5, .01],
        loss=['linear', 'square', 'exponential'],
        #random_state=scrambler[3:5]
        
    )],
        
    ['RandomForestRegressor', dict(
        max_depth=[5, 10, 15],
        criterion=['mse', 'mae'],
        #random_state=scrambler[:2]
    )],
    ['SVR', dict( #Most params for SVR are turned off right now, too expensive
        C=[1, .9],
        epsilon=[.1, .05],
        #kernel=['poly']
    )],
    ['GradientBoostingRegressor', dict(
        max_depth=[3, 6, 9, 12],
        min_samples_split=[2, 4, 8],
        presort=[False]
    )],
    ['LassoLars', dict(
        alpha=[0.1, 1, .5, .75],
        #random_state=[random.randint(0, 10000)]
    )],
    ]

[7965, 7888, 5716, 5270, 8053, 2626, 7813, 3195, 4649, 1960]


## Grid Search:

Here we implement an iterator that executes GridSearchCV and reports the best explained variance. The best_params attribute is then extracted, and used those on the whole training set, then predict on the holdout data.

Testing indicates that for some models, the fit on our full dataset modestly outperforms the CV regularly.

In [19]:
outcomes = []

for name, rgsr in zip(names, regressors):
    
    for item in param_grids:
        if item[0]==name:
            print(name + ':')
            params= item[1]
        
    
    cv = sklearn.model_selection.GridSearchCV(rgsr, param_grid=params,
                                              verbose=True, n_jobs=12,
                                              cv=3, pre_dispatch="2*n_jobs")
    
    if name not in to_scale:
        #X_train, y_train, X_test, y_test = sklearn.model_selection.train_test_split(X, y)
        fitted = cv.fit(XH_train, yH_train)
        score = cv.score(XH_val, yH_val)
        print(score)

        best = rgsr.set_params(**cv.best_params_)
        bestfit= best.fit(XH_train, yH_train)
        bestscore = best.score(XH_test, yH_test)
    if name in to_scale:
    #TODO: fix
        #X_train, y_train, X_test, y_test = sklearn.model_selection.train_test_split(sX, sy)
        fitted = cv.fit(sXH_train, syH_train)
        score = cv.score(sXH_val, syH_val)
        print(score)

        best = rgsr.set_params(**cv.best_params_)
        bestfit= best.fit(sXH_train, syH_train)
        bestscore = best.score(sXH_test, syH_test)

    print(name + " R2 with best model, score:")
    print(bestscore)
    
    outcomes.append((name, score, cv.cv_results_, cv.best_estimator_, 
                     cv.best_params_, bestscore, [yH_test, ]))
    
for nm in range(0, len(outcomes)):
    print()
    print(outcomes[nm][0])
    print(outcomes[nm][1])

    print()
    print('Best on real:')
    print(outcomes[nm][-1])
    
    

AdaBoostRegressor:
Fitting 3 folds for each of 27 candidates, totalling 81 fits


[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    4.8s
[Parallel(n_jobs=12)]: Done  81 out of  81 | elapsed:   13.2s finished


0.813438294075
AdaBoostRegressor R2 with best model, score:
0.62916562312
RandomForestRegressor:
Fitting 3 folds for each of 6 candidates, totalling 18 fits


[Parallel(n_jobs=12)]: Done  14 out of  18 | elapsed:    4.8s remaining:    1.4s
[Parallel(n_jobs=12)]: Done  18 out of  18 | elapsed:    5.2s finished


0.818558160246
RandomForestRegressor R2 with best model, score:
0.625537124872
SVR:
Fitting 3 folds for each of 4 candidates, totalling 12 fits


...

-0.279852405194
SVR R2 with best model, score:
-0.486678210057
GradientBoostingRegressor:
Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=12)]: Done  36 out of  36 | elapsed:   16.2s finished


0.811212323143
GradientBoostingRegressor R2 with best model, score:
0.643786101765
LassoLars:
Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=12)]: Done   2 out of  12 | elapsed:    0.1s remaining:    0.5s
[Parallel(n_jobs=12)]: Done  12 out of  12 | elapsed:    0.7s finished


0.838060151948
LassoLars R2 with best model, score:
0.840252467946

AdaBoostRegressor
0.813438294075

Best on real:
[1950     880.0
1951    1126.0
1952     935.0
1953     285.0
1954    2597.0
1955    1063.0
1956    1122.0
1957     480.0
1958     656.0
1959     636.0
1960     406.0
1961     374.0
1962     248.0
1963     398.0
1964     678.0
1965     667.0
1966     431.0
1967     381.0
1968     475.0
1969     836.0
1970     738.0
1971    1033.0
1972     888.0
1973    1197.0
1974    1052.0
1975     842.0
1976     859.0
1977     520.0
1978     378.0
1979     455.0
         ...  
2152     745.0
2153     513.0
2154     443.0
2155     534.0
2156    1069.0
2157     918.0
2158    1221.0
2159    1059.0
2160    1431.0
2161    1218.0
2162    1055.0
2163     971.0
2164     591.0
2165     458.0
2166     517.0
2167     500.0
2168     377.0
2169     650.0
2170     315.0
2171     538.0
2172     362.0
2173     485.0
2174    3903.0
2175    8204.0
2176    3139.0
2177    2645.0
2178    4185.0
2179    5717.

# Data Analysis - ANC 

In [20]:
data.corr()['y'].sort_values()

NAME_ANC 2D           -0.095615
NAME_ANC 3G           -0.090310
NAME_ANC 8B           -0.086048
NAME_ANC 1D           -0.085155
NAME_ANC 8E           -0.084710
NAME_ANC 7F           -0.083307
NAME_ANC 5A           -0.082504
NAME_ANC 8D           -0.081533
NAME_ANC 7E           -0.078977
NAME_ANC 7C           -0.078643
NAME_ANC 3F           -0.075906
NAME_ANC 4A           -0.075483
NAME_ANC 8C           -0.069964
NAME_ANC 4D           -0.069841
NAME_ANC 5B           -0.069615
NAME_ANC 3E           -0.068627
NAME_ANC 3B           -0.068553
NAME_ANC 7D           -0.065640
NAME_ANC 7B           -0.065575
Q_GDP                 -0.059282
NAME_ANC 3D           -0.059079
NAME_ANC 8A           -0.057420
NAME_ANC 3C           -0.054521
NAME_ANC 4B           -0.047929
NAME_ANC 4C           -0.045822
NAME_ANC 6E           -0.041732
NAME_ANC 5D           -0.035304
NAME_ANC 6D           -0.032935
NAME_ANC 5C           -0.027958
NAME_ANC 6A           -0.026828
                         ...   
NAME_ANC

In [21]:
#LassoLARS
blist = outcomes[-1] #this number is how we select which regressor
print(blist[0])
prms = LassoLars(**blist[4])
prms = prms.fit(sXH_train, syH_train)
print(prms)
print()
print('Score on test data:')
print(prms.score(XH_test, yH_test))
pred  = prms.predict(XH_test)
print()
print(prms.coef_path_) #Or whatever other attribute you want

LassoLars
LassoLars(alpha=0.1, copy_X=True, eps=2.2204460492503131e-16,
     fit_intercept=True, fit_path=True, max_iter=500, normalize=True,
     positive=False, precompute='auto', verbose=False)

Score on test data:
0.840252467946

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    9.09477160e+00   4.14197866e+01]
 [  0.00000000e+00   1.90816167e+04   1.98312885e+04 ...,   1.84317769e+04
    1.83865374e+04   1.82291905e+04]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.92571155e+02
    1.93232880e+02   1.95389590e+02]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.29262966e+02
    1.36782553e+02   1.55537306e+02]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,  -4.01128882e+01
   -4.22571169e+01  -5.40590021e+01]
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [22]:
lars_anc_beta = [i for i in zip(XH_test.columns, prms.coef_)]
sorted(lars_anc_beta, key=lambda x: x[1])

[('NAME_Ward 7', -9.9778053613997386),
 ('month', -5.4913680634256972),
 ('NAME_Ward 4', -2.3465232136179677),
 ('SALEPRICE', 0.0),
 ('Q_GDP', 0.0),
 ('PHARM_Concentr', 0.0),
 ('GROC_Concentr', 0.0),
 ('BANKS_Concentr', 0.0),
 ('CLUBS_Concentr', 0.0),
 ('HOTELS_Concentr', 0.0),
 ('METRO_Concentr', 0.0),
 ('countBBL_prev_month', 0.0),
 ('countBBL_prev_cycle', 0.0),
 ('NAME_ANC 1A', 0.0),
 ('NAME_ANC 1B', 0.0),
 ('NAME_ANC 1C', 0.0),
 ('NAME_ANC 1D', 0.0),
 ('NAME_ANC 2A', 0.0),
 ('NAME_ANC 2C', 0.0),
 ('NAME_ANC 2D', 0.0),
 ('NAME_ANC 2E', 0.0),
 ('NAME_ANC 3B', 0.0),
 ('NAME_ANC 3C', 0.0),
 ('NAME_ANC 3D', 0.0),
 ('NAME_ANC 3E', 0.0),
 ('NAME_ANC 3F', 0.0),
 ('NAME_ANC 3G', 0.0),
 ('NAME_ANC 4A', 0.0),
 ('NAME_ANC 4B', 0.0),
 ('NAME_ANC 4C', 0.0),
 ('NAME_ANC 4D', 0.0),
 ('NAME_ANC 5A', 0.0),
 ('NAME_ANC 5B', 0.0),
 ('NAME_ANC 5C', 0.0),
 ('NAME_ANC 5D', 0.0),
 ('NAME_ANC 5E', 0.0),
 ('NAME_ANC 6A', 0.0),
 ('NAME_ANC 6B', 0.0),
 ('NAME_ANC 6C', 0.0),
 ('NAME_ANC 6D', 0.0),
 ('NAME_ANC 

In [23]:
#AdaBoost
blist = outcomes[0] #this number is how we select which regressor
print(blist[0])
prms = AdaBoostRegressor(**blist[4])
prms = prms.fit(sXH_train, syH_train)
print(prms)
print()
print('Score on test data:')
print(prms.score(XH_test, yH_test))
pred  = prms.predict(XH_test)
print()
print(prms.feature_importances_) #Or whatever other attribute you want

AdaBoostRegressor
AdaBoostRegressor(base_estimator=None, learning_rate=1, loss='square',
         n_estimators=80, random_state=None)

Score on test data:
0.62916359784

[  4.02455389e-03   4.70016522e-01   0.00000000e+00   2.76160250e-03
   0.00000000e+00   4.36805176e-04   1.10266869e-02   1.30266264e-02
   2.61454822e-04   3.24354229e-03   3.13951946e-04   1.93840371e-03
   7.66771743e-04   1.42854206e-03   1.74735557e-03   2.22312382e-03
   1.64317021e-01   3.15894613e-01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
  

In [24]:
ada_anc_beta = [i for i in zip(XH_test.columns, prms.feature_importances_)]
sorted(ada_anc_beta, key=lambda x: x[1])

[('countIssued', 0.0),
 ('SALEPRICE', 0.0),
 ('NAME_ANC 1A', 0.0),
 ('NAME_ANC 1B', 0.0),
 ('NAME_ANC 1C', 0.0),
 ('NAME_ANC 1D', 0.0),
 ('NAME_ANC 2A', 0.0),
 ('NAME_ANC 2B', 0.0),
 ('NAME_ANC 2C', 0.0),
 ('NAME_ANC 2D', 0.0),
 ('NAME_ANC 2E', 0.0),
 ('NAME_ANC 2F', 0.0),
 ('NAME_ANC 3B', 0.0),
 ('NAME_ANC 3C', 0.0),
 ('NAME_ANC 3D', 0.0),
 ('NAME_ANC 3E', 0.0),
 ('NAME_ANC 3F', 0.0),
 ('NAME_ANC 3G', 0.0),
 ('NAME_ANC 4A', 0.0),
 ('NAME_ANC 4B', 0.0),
 ('NAME_ANC 4C', 0.0),
 ('NAME_ANC 4D', 0.0),
 ('NAME_ANC 5A', 0.0),
 ('NAME_ANC 5B', 0.0),
 ('NAME_ANC 5C', 0.0),
 ('NAME_ANC 5D', 0.0),
 ('NAME_ANC 5E', 0.0),
 ('NAME_ANC 6A', 0.0),
 ('NAME_ANC 6B', 0.0),
 ('NAME_ANC 6C', 0.0),
 ('NAME_ANC 6D', 0.0),
 ('NAME_ANC 6E', 0.0),
 ('NAME_ANC 7B', 0.0),
 ('NAME_ANC 7C', 0.0),
 ('NAME_ANC 7D', 0.0),
 ('NAME_ANC 7E', 0.0),
 ('NAME_ANC 7F', 0.0),
 ('NAME_ANC 8A', 0.0),
 ('NAME_ANC 8B', 0.0),
 ('NAME_ANC 8C', 0.0),
 ('NAME_ANC 8D', 0.0),
 ('NAME_ANC 8E', 0.0),
 ('NAME_Ward 1', 0.0),
 ('NAME_Ward 

In [25]:
#RFR 
blist = outcomes[1] #this number is how we select which regressor
print(blist[0])
prms = RandomForestRegressor(**blist[4])
prms = prms.fit(sXH_train, syH_train)
print(prms)
print()
print('Score on test data:')
print(prms.score(XH_test, yH_test))
pred  = prms.predict(XH_test)
print()
print(prms.feature_importances_) #Or whatever other attribute you want

RandomForestRegressor
RandomForestRegressor(bootstrap=True, criterion='mae', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

Score on test data:
0.652522770273

[  3.01361205e-02   5.97960388e-01   3.37107834e-03   1.13620832e-02
   3.88125885e-03   2.54736835e-03   4.59641016e-03   2.88645736e-03
   2.04787575e-03   1.77534402e-03   1.22303326e-03   9.68791443e-04
   6.43739234e-04   2.01442529e-03   3.40219819e-03   1.99849678e-03
   2.04424365e-01   1.19556205e-01   2.24332587e-05   7.59619148e-05
   4.00051334e-04   5.02046737e-06   0.00000000e+00   6.51576634e-06
   0.00000000e+00   2.49230344e-04   4.56585108e-05   6.88045993e-04
   9.94429608e-06   2.51735282e-05   0.00000000e+00   6.83127215e-05
   1.88661782e-05   3.19351063e-05

In [26]:
rfr_anc_beta = [i for i in zip(XH_test.columns, prms.feature_importances_)]
sorted(rfr_anc_beta, key=lambda x: x[1])

[('NAME_ANC 2A', 0.0),
 ('NAME_ANC 2C', 0.0),
 ('NAME_ANC 3D', 0.0),
 ('NAME_ANC 8B', 3.2562565351259585e-06),
 ('NAME_ANC 6B', 4.1231488784393137e-06),
 ('NAME_ANC 1D', 5.0204673696516328e-06),
 ('NAME_ANC 2B', 6.5157663446188815e-06),
 ('NAME_ANC 3B', 9.944296077858284e-06),
 ('NAME_ANC 5A', 1.3289904221051161e-05),
 ('NAME_ANC 3F', 1.8866178242103493e-05),
 ('NAME_ANC 5B', 1.9825897238674587e-05),
 ('NAME_ANC 7D', 2.0088839774876836e-05),
 ('NAME_Ward 5', 2.1840423910864691e-05),
 ('NAME_ANC 1A', 2.2433258749762986e-05),
 ('NAME_ANC 3C', 2.5173528223375042e-05),
 ('NAME_ANC 3G', 3.1935106274377426e-05),
 ('NAME_ANC 4A', 3.252030242421655e-05),
 ('NAME_ANC 7E', 3.4769305842010211e-05),
 ('NAME_ANC 6E', 4.1908497652242317e-05),
 ('NAME_ANC 4B', 4.353921255449712e-05),
 ('NAME_ANC 8E', 4.3790584941875907e-05),
 ('NAME_ANC 2E', 4.5658510774055832e-05),
 ('NAME_ANC 7C', 4.7690686860568231e-05),
 ('NAME_ANC 8A', 5.6861650323590474e-05),
 ('NAME_ANC 6C', 5.718147451615851e-05),
 ('NAME_ANC

In [27]:
#GBR  
blist = outcomes[3] #this number is how we select which regressor
print(blist[0])
prms = GradientBoostingRegressor(**blist[4])
prms = prms.fit(sXH_train, syH_train)
print(prms)
print()
print('Score on test data:')
print(prms.score(XH_test, yH_test))
pred  = prms.predict(XH_test)
print()
print(prms.feature_importances_) #Or whatever other attribute you want

GradientBoostingRegressor
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_split=1e-07,
             min_samples_leaf=1, min_samples_split=4,
             min_weight_fraction_leaf=0.0, n_estimators=100, presort=False,
             random_state=None, subsample=1.0, verbose=0, warm_start=False)

Score on test data:
0.622907828703

[ 0.04267466  0.25549895  0.04992259  0.15322609  0.01460395  0.04650302
  0.02187223  0.01729032  0.00762472  0.00180342  0.00275576  0.00548236
  0.00815226  0.00840821  0.00248668  0.02472876  0.12715914  0.12798261
  0.          0.00259045  0.          0.          0.          0.          0.
  0.00190082  0.00355976  0.01266555  0.          0.          0.00225569
  0.          0.          0.          0.          0.00150033  0.00438434
  0.          0.          0.          0.          0.          0.0127854   0.
  0

In [28]:
gbr_anc_beta = [i for i in zip(XH_test.columns, prms.feature_importances_)]
sorted(gbr_anc_beta, key=lambda x: x[1])

[('NAME_ANC 1A', 0.0),
 ('NAME_ANC 1C', 0.0),
 ('NAME_ANC 1D', 0.0),
 ('NAME_ANC 2A', 0.0),
 ('NAME_ANC 2B', 0.0),
 ('NAME_ANC 2C', 0.0),
 ('NAME_ANC 3B', 0.0),
 ('NAME_ANC 3C', 0.0),
 ('NAME_ANC 3E', 0.0),
 ('NAME_ANC 3F', 0.0),
 ('NAME_ANC 3G', 0.0),
 ('NAME_ANC 4A', 0.0),
 ('NAME_ANC 4D', 0.0),
 ('NAME_ANC 5A', 0.0),
 ('NAME_ANC 5B', 0.0),
 ('NAME_ANC 5C', 0.0),
 ('NAME_ANC 5D', 0.0),
 ('NAME_ANC 6A', 0.0),
 ('NAME_ANC 6B', 0.0),
 ('NAME_ANC 6C', 0.0),
 ('NAME_ANC 7C', 0.0),
 ('NAME_ANC 7D', 0.0),
 ('NAME_ANC 7E', 0.0),
 ('NAME_ANC 7F', 0.0),
 ('NAME_ANC 8A', 0.0),
 ('NAME_ANC 8B', 0.0),
 ('NAME_ANC 8D', 0.0),
 ('NAME_ANC 8E', 0.0),
 ('NAME_Ward 4', 0.0),
 ('NAME_Ward 5', 0.0),
 ('NAME_Ward 7', 0.0),
 ('NAME_Ward 8', 0.0),
 ('NAME_ANC 7B', 0.00093781561136438581),
 ('NAME_ANC 6D', 0.0014989498543691651),
 ('NAME_ANC 4B', 0.0015003332776836837),
 ('PHARM_Concentr', 0.0018034182658250297),
 ('NAME_ANC 2D', 0.0019008154389531856),
 ('NAME_ANC 3D', 0.0022556881410009193),
 ('METRO_Conce

# Charts

In [ ]:
#Adapted from https://pythonspot.com/en/matplotlib-bar-chart/

objects     = [j[0] for j in outcomes]
y_pos       = np.arange(len(objects))

performance = [j[-1] for j in outcomes]
for jm in range(len(performance)):
    if performance[jm] < 0:
        performance[jm] = 0
performance

In [ ]:
plt.barh(y_pos, performance, align='center', alpha=0.5)
plt.yticks(y_pos, objects)
plt.xlabel('R2 Score')
ti = "Scoring across models for "+shapef+", lagging by "+str(shiftmonths)+ " months."
plt.title(ti)
fl = './plots/' + shapef + "_shift" + str(shiftmonths)
plt.savefig(fl)

# Everything below is exploratory analysis for me.

In [ ]:
for jm in range(0, 5):
    
    print(outcomes[jm][0])
    
    print(outcomes[jm][1])
    print(outcomes[jm][4])

In [ ]:
best = AdaBoostRegressor(learning_rate=1, loss='square', n_estimators=60)
bestfit= best.fit(XH_train, yH_train)
bestscore = best.score(XH_test, yH_test)
print(outcomes[0][0])
print(bestscore)

best = RandomForestRegressor(max_depth=10)
bestfit= best.fit(XH_train, yH_train)
bestscore = best.score(XH_test, yH_test)
print(outcomes[1][0])
print(bestscore)

best = SVR(max_depth=10)
bestfit= best.fit(XH_train, yH_train)
bestscore = best.score(XH_test, yH_test)
print(outcomes[2][0])
print(bestscore)


best = GradientBoostingRegressor(max_depth=10)
bestfit= best.fit(XH_train, yH_train)
bestscore = best.score(XH_test, yH_test)
print(outcomes[3][0])
print(bestscore)

In [ ]:


Xtrain = dat_xtrain.drop(['y'], axis=1)
y16 = dat_ytrain['y']
X15 = dat15.drop(['y'], axis=1)
y15 = dat15['y']

fitted    = outcomes[-2][3].fit(X15, y15)
predicted = fitted.predict(X16)

In [ ]:
pred = pd.DataFrame(predicted, columns=['predicted'])
dat16 = dat16.reset_index()
pred['y'] = dat16['y']

In [ ]:
def flagger_ranges(pred):
    pred['flag15'] = 0
    pred['flag15'][pred['predicted'].between(pred['y']*0.85, pred['y']*1.15)
                  ] = 1
    pred['flag05'] = 0
    pred['flag05'][pred['predicted'].between(pred['y']*0.85, pred['y']*1.15)
                  ] = 1
    pred['flag10'] = 0
    pred['flag10'][pred['predicted'].between(pred['y']*0.85, pred['y']*1.15)
                  ] = 1
    pred['flag_others']= 0
    pred['flag_others'][pred['flag05'] == 0] = 1
    return pred
pred = flagger_ranges(pred)
pred